# 01 - Exploración de Datos

En este notebook vamos hacer la carga de los datos desde el archivo `train.csv` que encontramos en la competencia de Kaggle UDEA/ai4eng 20252

Exploramos inicialmente la estructura de los datos, las distintas variables disponibles, la cantidad de registros y algunas distribuciones básicas.

### Realizamos las importaciones necesarias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Traemos los dataset desde Kaggle

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20252